#### Contact Kickstarter and get permission before crawling their website
##### Email - support@kickstarter.com

In [2]:
from selenium import webdriver
import regex as re
import numpy as np
import pandas as pd
from dask import delayed
import time
import os
import sys
import glob
from time import sleep

## Gathered initial data from url -

https://webrobots.io/kickstarter-datasets/ 

### Appended individual data files to put all the data in one

In [3]:
kick = pd.DataFrame()

for file in glob.glob("C:\OSU\Sem3\BAN 5753\Kickstarter_2018-09-13T03_20_17_777Z\*"):
    df = pd.read_csv(file)
    kick = kick.append(df)
    print("File appended: " + file)

File appended: C:\OSU\Sem3\BAN 5753\SAS Symposium\Kickstarter_2018-09-13T03_20_17_777Z\Kickstarter.csv
File appended: C:\OSU\Sem3\BAN 5753\SAS Symposium\Kickstarter_2018-09-13T03_20_17_777Z\Kickstarter001.csv
File appended: C:\OSU\Sem3\BAN 5753\SAS Symposium\Kickstarter_2018-09-13T03_20_17_777Z\Kickstarter002.csv
File appended: C:\OSU\Sem3\BAN 5753\SAS Symposium\Kickstarter_2018-09-13T03_20_17_777Z\Kickstarter003.csv
File appended: C:\OSU\Sem3\BAN 5753\SAS Symposium\Kickstarter_2018-09-13T03_20_17_777Z\Kickstarter004.csv
File appended: C:\OSU\Sem3\BAN 5753\SAS Symposium\Kickstarter_2018-09-13T03_20_17_777Z\Kickstarter005.csv
File appended: C:\OSU\Sem3\BAN 5753\SAS Symposium\Kickstarter_2018-09-13T03_20_17_777Z\Kickstarter006.csv
File appended: C:\OSU\Sem3\BAN 5753\SAS Symposium\Kickstarter_2018-09-13T03_20_17_777Z\Kickstarter007.csv
File appended: C:\OSU\Sem3\BAN 5753\SAS Symposium\Kickstarter_2018-09-13T03_20_17_777Z\Kickstarter008.csv
File appended: C:\OSU\Sem3\BAN 5753\SAS Symposium

In [4]:
kick.reset_index(inplace= True)

### just need urls of different projects that we can use further to scrape the data using selenium

In [5]:
kick_urls = kick[['id','urls']]

In [6]:
kick_urls.head()

,id,urls
0,62545913,"{""web"":{""project"":""https://www.kickstarter.com..."
1,2031730466,"{""web"":{""project"":""https://www.kickstarter.com..."
2,55673621,"{""web"":{""project"":""https://www.kickstarter.com..."
3,614137516,"{""web"":{""project"":""https://www.kickstarter.com..."
4,1164271753,"{""web"":{""project"":""https://www.kickstarter.com..."


### clean the urls column using regex in python to get urls in consistent form

In [7]:
%%time

kick_urls_final = pd.DataFrame()

start_time = time.time()

for row in range(len(kick_urls)):
    id = kick_urls['id'][row]
    
    y=re.findall('"project"\:"http\S*,"',kick_urls.urls[row])
    y = y[0][11:len(y[0])-3]
    str1 = ''.join(y)
    d = {"id":[id], "urls":[str1]}
    df = pd.DataFrame(data=d)
    
    kick_urls_final = kick_urls_final.append(df)
    
kick_urls_final.reset_index(inplace=True, drop=True)

print("--- %s hours ---" % str((time.time() - start_time)/3600))

--- 0.4798923964632882 hours ---
Wall time: 28min 47s


In [8]:
print(kick_urls_final.count())
kick_urls_final.head()

id      205091
urls    205091
dtype: int64


,id,urls
0,62545913,https://www.kickstarter.com/projects/101710808...
1,2031730466,https://www.kickstarter.com/projects/270084862...
2,55673621,https://www.kickstarter.com/projects/lukewilso...
3,614137516,https://www.kickstarter.com/projects/119048645...
4,1164271753,https://www.kickstarter.com/projects/679958171...


## Used selenium with chrome to scrape Kickstarter projects description 

In [56]:
kickstarter = pd.DataFrame()

In [57]:
%%time

start_time = time.time()

driver = webdriver.Chrome(executable_path=r'C:/chromedriver.exe')

for row in range(0, 1000):
    
    project_id = kick_urls_final["id"][row]
    url = kick_urls_final["urls"][row]
    
    try:
        driver.get(url)
    except:
        pass
        
    #time.sleep(20)
    try:   
        desc = driver.find_element_by_css_selector("#content-wrap > div.NS_projects__content.pt11 > section.js-project-content.js-project-description-content.project-content > div > div > div > div > div.col.col-8.description-container > div.full-description.js-full-description.responsive-media.formatted-lists")
        des = desc.text
        
        #if des == np.nan:
        #    des = ''
            
    except:
        pass
    
    #try:
    #    risk = driver.find_element_by_css_selector("#content-wrap > div.NS_projects__content.pt11 > section.js-project-content.js-project-description-content.project-content > div > div > div > div > div.col.col-8.description-container > div.mb3.mb10-sm.mb3.js-risks")
    #    ris = risk.text
    #    
    #    if ris == np.nan:
    #        ris = ''
            
    #except:
    #    continue

    #d1 = {"id":[project_id],"url":[url],"description":[des],"risk and challenges":[ris[21:]]}
    d1 = {"id":[project_id],"url":[url],"description":[des]}

    df1 = pd.DataFrame(data = d1)

    kickstarter = kickstarter.append(df1)

    print(str(row) + " project(s) are done")

    des = ''
    ris = ''
    
    if row == 999:
        driver.close()
    
#kickstarter_updated = kickstarter[['id', 'url', 'description', 'risk and challenges']]
kickstarter_updated = kickstarter[['id', 'url', 'description']]

print("--- %s hours ---" % str((time.time() - start_time)/3600))

0 project(s) are done
1 project(s) are done
2 project(s) are done
3 project(s) are done
4 project(s) are done
5 project(s) are done
6 project(s) are done
7 project(s) are done
8 project(s) are done
9 project(s) are done
10 project(s) are done
11 project(s) are done
12 project(s) are done
13 project(s) are done
14 project(s) are done
15 project(s) are done
16 project(s) are done
17 project(s) are done
18 project(s) are done
19 project(s) are done
20 project(s) are done
21 project(s) are done
22 project(s) are done
23 project(s) are done
24 project(s) are done
25 project(s) are done
26 project(s) are done
27 project(s) are done
28 project(s) are done
29 project(s) are done
30 project(s) are done
31 project(s) are done
32 project(s) are done
33 project(s) are done
34 project(s) are done
35 project(s) are done
36 project(s) are done
37 project(s) are done
38 project(s) are done
39 project(s) are done
40 project(s) are done
41 project(s) are done
42 project(s) are done
43 project(s) are don

346 project(s) are done
347 project(s) are done
348 project(s) are done
349 project(s) are done
350 project(s) are done
351 project(s) are done
352 project(s) are done
353 project(s) are done
354 project(s) are done
355 project(s) are done
356 project(s) are done
357 project(s) are done
358 project(s) are done
359 project(s) are done
360 project(s) are done
361 project(s) are done
362 project(s) are done
363 project(s) are done
364 project(s) are done
365 project(s) are done
366 project(s) are done
367 project(s) are done
368 project(s) are done
369 project(s) are done
370 project(s) are done
371 project(s) are done
372 project(s) are done
373 project(s) are done
374 project(s) are done
375 project(s) are done
376 project(s) are done
377 project(s) are done
378 project(s) are done
379 project(s) are done
380 project(s) are done
381 project(s) are done
382 project(s) are done
383 project(s) are done
384 project(s) are done
385 project(s) are done
386 project(s) are done
387 project(s) a

688 project(s) are done
689 project(s) are done
690 project(s) are done
691 project(s) are done
692 project(s) are done
693 project(s) are done
694 project(s) are done
695 project(s) are done
696 project(s) are done
697 project(s) are done
698 project(s) are done
699 project(s) are done
700 project(s) are done
701 project(s) are done
702 project(s) are done
703 project(s) are done
704 project(s) are done
705 project(s) are done
706 project(s) are done
707 project(s) are done
708 project(s) are done
709 project(s) are done
710 project(s) are done
711 project(s) are done
712 project(s) are done
713 project(s) are done
714 project(s) are done
715 project(s) are done
716 project(s) are done
717 project(s) are done
718 project(s) are done
719 project(s) are done
720 project(s) are done
721 project(s) are done
722 project(s) are done
723 project(s) are done
724 project(s) are done
725 project(s) are done
726 project(s) are done
727 project(s) are done
728 project(s) are done
729 project(s) a

### above code would scrape data for first 1000 projects
### change row == number in the if loop at the end of the code to extract data for more projects

In [58]:
kickstarter_updated.count()

id             1000
url            1000
description    1000
dtype: int64

In [59]:
kickstarter_updated.reset_index(inplace=True, drop=True)
kickstarter_updated.head(100)

,id,url,description
0,62545913,https://www.kickstarter.com/projects/101710808...,**We reached our goal thanks to your help! But...
1,2031730466,https://www.kickstarter.com/projects/270084862...,"This book doesn't just talk about an idea, Sco..."
2,55673621,https://www.kickstarter.com/projects/lukewilso...,Stretch Goals!\nThanks to all of our great fri...
3,614137516,https://www.kickstarter.com/projects/119048645...,The Longest Lie I Ever Told\nLying is an under...
4,1164271753,https://www.kickstarter.com/projects/679958171...,Meine Masterarbeit in Philosophie bemüht sich ...
5,241806675,https://www.kickstarter.com/projects/194429923...,Finally! The long awaited final volume in the...
6,384486589,https://www.kickstarter.com/projects/thepenn/t...,We're hiking 272-miles of the Long Trail and w...
7,655402823,https://www.kickstarter.com/projects/withinrea...,"HEY, I DON’T EVEN UNDERSTAND THE TITLE!\n\nAko..."
8,2145910362,https://www.kickstarter.com/projects/TravisEne...,It All Started with a Question... \nHi! My na...
9,2102229549,https://www.kickstarter.com/projects/worldride...,"A Quest For Culture, Cuisine and Connection.\n..."


### data saved in a csv file on local drive

In [60]:
kickstarter_updated.to_csv("C:\OSU\Sem3\BAN 5753\kick_sample.csv", index = False)

## Extra practice with Selenium using python

In [3]:
driver = webdriver.Chrome(executable_path=r'C:/chromedriver.exe')
driver.get("https://www.kickstarter.com/projects/101710808/how-weve-changed-jesus-neal-samudres-debut-book?ref=category_newest")
el=driver.find_element_by_tag_name("body")
print(el.text)

Kickstarter
Explore
Start a project
Search
Sign in
"How We've Changed Jesus" - Neal Samudre's Debut Book
If you've ever been disheartened by how some people might represent the character of Christ, this is the book for you.
Created by
Neal Samudre
80 backers pledged $3,651 to help bring this project to life.
Campaign FAQ Updates 12 Comments 1 Community
About
PLAY

Harrisonburg, VA
Nonfiction
$3,651
pledged of $3,500 goal
80
backers
**We reached our goal thanks to your help! But please continue to pledge as any extra money over my goal will go towards production costs, allowing me to get the book in more people's hands!**

THE STORY.
This is a book for doubters and believers, but mainly doubters. If you're anything like me, at some point in time, you've been disheartened by how people sometimes represent Christ. You've had different portrayals of Jesus given to you. But how do you know which is the true picture of Jesus? Somewhere in the mess of this noisy world is the true picture of J

In [5]:
driver = webdriver.Chrome(executable_path=r'C:/chromedriver.exe')
driver.get("https://www.kickstarter.com/projects/101710808/how-weve-changed-jesus-neal-samudres-debut-book?ref=category_newest")
el=driver.find_element_by_tag_name("main")
print(el.text)

"How We've Changed Jesus" - Neal Samudre's Debut Book
If you've ever been disheartened by how some people might represent the character of Christ, this is the book for you.
Created by
Neal Samudre
80 backers pledged $3,651 to help bring this project to life.
Campaign FAQ Updates 12 Comments 1 Community
About
PLAY

Harrisonburg, VA
Nonfiction
$3,651
pledged of $3,500 goal
80
backers
**We reached our goal thanks to your help! But please continue to pledge as any extra money over my goal will go towards production costs, allowing me to get the book in more people's hands!**

THE STORY.
This is a book for doubters and believers, but mainly doubters. If you're anything like me, at some point in time, you've been disheartened by how people sometimes represent Christ. You've had different portrayals of Jesus given to you. But how do you know which is the true picture of Jesus? Somewhere in the mess of this noisy world is the true picture of Jesus, and with the help of Scripture and personal e

In [8]:
driver = webdriver.Chrome(executable_path=r'C:/chromedriver.exe')
driver.get("https://www.kickstarter.com/projects/101710808/how-weve-changed-jesus-neal-samudres-debut-book?ref=category_newest")
el=driver.find_element_by_tag_name("p")
print(el.text)

This is a book for doubters and believers, but mainly doubters. If you're anything like me, at some point in time, you've been disheartened by how people sometimes represent Christ. You've had different portrayals of Jesus given to you. But how do you know which is the true picture of Jesus? Somewhere in the mess of this noisy world is the true picture of Jesus, and with the help of Scripture and personal experiences, we will discover together how Jesus should really be represented on this Earth.


In [15]:
driver = webdriver.Chrome(executable_path=r'C:/chromedriver.exe')
driver.get("https://www.kickstarter.com/projects/101710808/how-weve-changed-jesus-neal-samudres-debut-book?ref=category_newest")
el=driver.find_element_by_css_selector("#content-wrap > div.NS_projects__content.pt11 > section.js-project-content.js-project-description-content.project-content > div > div > div > div > div.col.col-8.description-container > div.full-description.js-full-description.responsive-media.formatted-lists")
print(el.text)

**We reached our goal thanks to your help! But please continue to pledge as any extra money over my goal will go towards production costs, allowing me to get the book in more people's hands!**

THE STORY.
This is a book for doubters and believers, but mainly doubters. If you're anything like me, at some point in time, you've been disheartened by how people sometimes represent Christ. You've had different portrayals of Jesus given to you. But how do you know which is the true picture of Jesus? Somewhere in the mess of this noisy world is the true picture of Jesus, and with the help of Scripture and personal experiences, we will discover together how Jesus should really be represented on this Earth.
In "How We've Changed Jesus," each chapter will present a picture of Jesus that I encountered somewhere in my life. Examples include: The Hipster Jesus, The Political Jesus, The Slandering Jesus, and The Extroverted Jesus, and many more!
Partner with me on this mission and we'll show that wit

In [16]:
driver = webdriver.Chrome(executable_path=r'C:/chromedriver.exe')
driver.get("https://www.kickstarter.com/projects/270084862/your-client-vision-book-how-to-sell-without-sellin?ref=category_newest")
el=driver.find_element_by_css_selector("#content-wrap > div.NS_projects__content.pt11 > section.js-project-content.js-project-description-content.project-content > div > div > div > div > div.col.col-8.description-container > div.full-description.js-full-description.responsive-media.formatted-lists")
print(el.text)

This book doesn't just talk about an idea, Scott reveals HOW to attract your ideal client and implement a lead to sales conversion strategy. Because selling your services in todays world has changed. No longer can you hard sell or use clever sales techniques. 
Scott is an inspirational marketing mentor who has added over £30m turnover to clients he has worked with.
In this book Scott goes through in detail the three key parts of his marketing strategy that puts him, and his numerous clients, in the position of never having to sell himself or his services.
This strategy enhances the relationships with his clients so that they are no longer 'transactional' relationships and as a result they pay for services again and again and again without Scott ever selling.
YOUR CLIENT VISION is where all your marketing should start from.  Imagine someone asking you "what do you do?" and when you reply they say to you "we need to have a chat."  This is what your client vision does, and in such a power